In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.linear_model import LinearRegression
import scipy.stats as stats
import datetime as dt
import decimal

import sys
sys.path.insert(0, "../src/")
import util as util

%autosave 5

In [ ]:
df = pd.read_csv('../Data/EvolyticsDataScienceExercise.csv')
revenueCols = [col for col in df.columns if 'purchase_revenue' in str(col)]
revenuedf = (df[revenueCols])
for col in revenueCols:
    revenuedf[col] = pd.to_numeric(revenuedf[col],errors='coerce')
df['orderRevenue'] = revenuedf.sum(axis=1)

In [ ]:
revenuedf

# Univariate Analysis

In [ ]:
datetimelikecolumns = ['date','min_timestamp_date','min_timestamp_time','max_timestamp','date.1','upsell_timestamp_date','upsell_timestamp_time']
catagoricalCols = []

n = df.shape[0]
for col in df.columns:
    col = str(col)
    print(col)
    parsedValues = [util.tryParse(x)[1] for x in df[col] if util.tryParse(x)[0]]
    
    if col in datetimelikecolumns:
        sample = df[df[col] != "(null)"].dropna()[col]
        sample =  [x for x in pd.to_datetime(sample)]
        n_nans = n - len(sample)
        print(f'{n_nans} nans found. {n_nans*100.0/n}% of data is nan')
        fig = plt.figure()
        plt.hist(sample)
        plt.xticks(rotation=90)
        plt.show()
    elif (col != 'user_State' and len(parsedValues) > 0 ) and (len(list(set(df[col]))) > 30):
        sample = np.array(parsedValues)
        n_nans = n - len(sample)
        print(f'{n_nans} nans found. {n_nans*100.0/n}% of data is nan')
        print(f'{np.mean(sample)} +/- {np.std(sample)}')
        fig = plt.figure()
        plt.hist(sample)
        plt.show()
    else:
        catagoricalCols.append(col)
        print(df[col].value_counts())
    

# Correlation Analysis

In [ ]:
corrDictionary = {
    "name":[],
    "iCol":[],
    "jCol":[],
    "r":[],
    "absr":[],
    "p":[],
    "m":[],
    "b":[]
}
dfCopy = df.copy()
for dtcol in datetimelikecolumns:
    dfCopy[dtcol] = (pd.to_datetime(dfCopy[dtcol],errors='coerce') - dt.datetime(1970,1,1)).dt.total_seconds()

for catcol in catagoricalCols:
    dummyDF = pd.get_dummies(dfCopy[catcol])
    pd.concat([dfCopy,dummyDF],axis = 1)
    #for dummyCol in dummyDF.columns:
        #dfCopy[dummyCol] = dummyDF[dummyCol]
    dfCopy.drop(catcol,axis=1,inplace=True)
    
for i,iCol in enumerate(dfCopy.columns):
    if dfCopy[iCol].dtype == '<U38':
        dfCopy[iCol] = dfCopy[iCol].apply(lambda x: float(decimal.Decimal(x)))
    for j,jCol in enumerate(dfCopy.columns):
        if dfCopy[jCol].dtype == '<U38':
            dfCopy[jCol] = dfCopy[jCol].apply(lambda x: float(decimal.Decimal(x)))
        if i < j:
            corrDF = dfCopy[[iCol,jCol]].dropna()
            if corrDF.shape[0] > 0:
                iList = [util.tryParse(x)[1] for x in corrDF[iCol]]
                jList = [util.tryParse(y)[1] for y in corrDF[jCol]]
                r,p = stats.spearmanr(iList,jList)
                if p < 0.05 and abs(r) >= .1:
                    corrDictionary["name"].append(f"{jCol} vs {iCol}")
                    corrDictionary["iCol"].append(iCol) 
                    corrDictionary["jCol"].append(jCol) 
                    corrDictionary["r"].append(r)
                    corrDictionary["absr"].append(abs(r))
                    corrDictionary["p"].append(p)
                    lm = LinearRegression()
                    lm.fit(np.array(corrDF[iCol]).reshape(-1,1),np.array(corrDF[jCol]).reshape(-1,1))
                    corrDictionary["m"].append(lm.coef_[0][0])
                    corrDictionary["b"].append(lm.intercept_[0])
                

In [ ]:
corrDF = pd.DataFrame(corrDictionary,columns=list(corrDictionary.keys())).sort_values(by='absr',ascending=False)

In [ ]:
for i in range(corrDF.shape[0]):
    name = corrDF.iloc[i,corrDF.columns.get_loc('name')]
    iCol = corrDF.iloc[i,corrDF.columns.get_loc('iCol')]
    jCol = corrDF.iloc[i,corrDF.columns.get_loc('jCol')]
    r = corrDF.iloc[i,corrDF.columns.get_loc('r')]
    m = corrDF.iloc[i,corrDF.columns.get_loc('m')]
    b = corrDF.iloc[i,corrDF.columns.get_loc('b')]
    p = corrDF.iloc[i,corrDF.columns.get_loc('p')]
    fig = plt.figure()
    plotDF = dfCopy[[iCol,jCol]].dropna()
    iData = [util.tryParse(x)[1] for x in plotDF[iCol]]
    jData = [util.tryParse(y)[1] for y in plotDF[jCol]]
    plt.scatter(iData,jData)
    x = np.linspace(min(iData),max(iData))
    y = m * x + b
    plt.plot(x,y)
    plt.title(f'{name} R={r} p={p}')
    plt.xlabel(iCol)
    plt.xticks(rotation=90)
    plt.ylabel(jCol)
    plt.show()

In [ ]:
corrDF

# Conversion Analysis

In [ ]:
uniqueUsers = list(set(df['source_visitor_id']))
conversionDictionary = {
    'user': [],
    'experimentalVisits': [],
    'ordersMade': [],
    'totalRevenueFromUser':[]
}
for user in uniqueUsers:
    conversionDictionary['user'].append(user)
    userDF = df[df['source_visitor_id'] == user]
    conversionDictionary['experimentalVisits'].append(userDF.shape[0])
    conversionDictionary['ordersMade'].append(userDF['purchase_flag'].sum())
    conversionDictionary['totalRevenueFromUser'].append(userDF['orderRevenue'].sum())
conversionDF = pd.DataFrame(conversionDictionary,columns=list(conversionDictionary.keys()))
conversionDF['conversionRate'] = conversionDF['ordersMade']/conversionDF['experimentalVisits']

In [ ]:
print(f"Conversion Rate per User: {conversionDF['conversionRate'].mean()} +/- {conversionDF['conversionRate'].std()/np.sqrt(userDF.shape[0])}")
print(f"Average Total Revenue per User: {conversionDF['totalRevenueFromUser'].mean()} +/- {conversionDF['totalRevenueFromUser'].std()/np.sqrt(userDF.shape[0])}")

In [ ]:
conversionDF['experimentalVisits'].value_counts()

In [ ]:
conversionDF['ordersMade'].value_counts()

# Revenue Per Order Analysis

In [ ]:
print(f"Average Revenue per Order: {df['orderRevenue'].mean()} +/- {df['orderRevenue'].std()/np.sqrt(df.shape[0])}")